In [1]:
import snappy

We load the surgery descriptions of the knots and verify their hyperbolicity.

In [2]:
LB=snappy.Manifold('LB.lnk')
LB.verify_hyperbolicity()[0]

True

In [3]:
LG=snappy.Manifold('LG.lnk')
LG.verify_hyperbolicity()[0]

True

First we confirm (for small values of n) that the surgeries are actually isometric. And in a second step we prove that the knots are not isotopic.

In [4]:
def all_positive(manifold):
    '''
    Checks if the solution type of a triangulation is positive.
    '''
    return manifold.solution_type() == 'all tetrahedra positively oriented'

def find_positive_triangulations(manifold,number=1,tries=100):
    '''
    Searches for one triangulation with a positive solution type.
    (Or if number is set to a different value also for different such triangulations.)
    '''
    M = manifold.copy()
    pos_triangulations=[]
    for i in range(tries):
        if all_positive(M):
            pos_triangulations.append(M)
            if len(pos_triangulations)==number:
                return pos_triangulations
            break
        M.randomize()
    for d in M.dual_curves(max_segments=500):
        X = M.drill(d)
        X = X.filled_triangulation()
        X.dehn_fill((1,0),-1)
        for i in range(tries):
            if all_positive(X):
                pos_triangulations.append(X)
                if len(pos_triangulations)==number:
                    return pos_triangulations
                break
            X.randomize()

    # In the closed case, here is another trick.
    if all(not c for c in M.cusp_info('is_complete')):
        for i in range(tries):
            # Drills out a random edge
            X = M.__class__(M.filled_triangulation())
            if all_positive(X):
                pos_triangulations.append(X)
                if len(pos_triangulations)==number:
                    return pos_triangulations
            break
            M.randomize()
    return pos_triangulations

def better_volume(M,index=100,try_hard=False):
    '''Computes the verified volume. Returns 0 if SnapPy could not do it.'''
    count=0
    while count<index:
        try:
            return M.volume(verified=True)
        except:
            M.randomize()
            count=count+1
    if try_hard==True:
        pos_triang=find_positive_triangulations(M,number=1,tries=index)
        for X in pos_triang:
            vol=better_volume(X,index)
            if vol!=0:
                return vol
    return 0

def better_is_isometric_to(X,Y,index=100):
    """
    Returns True if X and Y are isometric.
    Returns False if X and Y have different homologies.
    Returns 'unclear' if SnapPy cannot verify it.
    The higher the index the harder SnapPy tries.
    """     
    w='unclear'
    if X.homology()!=Y.homology():
        w=False
    if w=='unclear':
        for i in (0,index):
            try:
                w=X.is_isometric_to(Y)
            except RuntimeError:
                pass
            except snappy.SnapPeaFatalError:
                pass
            if w==True:
                break
            if w==False:
                w='unclear'
            X.randomize()
            Y.randomize()
            i=i+1
    return w

In [5]:
for n in range(1,21):
    LB.dehn_fill([(-7-2*n,1),(-1,n)])
    LG.dehn_fill([(-3,1),(-1,n),(1,2*n+4)])
    print(n,better_is_isometric_to(LB,LG))

1 True
2 True
3 True
4 True
5 True
6 True
7 True
8 True
9 True
10 True
11 True
12 True
13 True
14 True
15 True
16 True
17 True
18 True
19 True
20 True


To complete the proof of the theorem, we show that the resulting knots K_B_n and K_G_n are different for all n. For that we compute the volumes of the knots using our surgery descriptions. And use the result that the volumes of the families converge monotonically to the volume of the surgery link. So we first compute the volumes of the surgery links and then check how the volumes of the families converge to this value.

In [6]:
LB.dehn_fill([(0,0),(0,0)])
better_volume(LB)

13.3503910775?

In [7]:
LG.dehn_fill([(0,0),(0,0),(0,0)])
better_volume(LG)

15.0353797930?

In [8]:
for n in range(1,20):
    LB.dehn_fill([(0,0),(-1,n)])
    LG.dehn_fill([(0,0),(-1,n),(1,2*n+4)])
    print('n=',n)
    print('Volume of K_B_n=',better_volume(LB))
    print('Volume of K_G_n=',better_volume(LG))

n= 1
Volume of K_B_n= 9.2100390724?
Volume of K_G_n= 10.21419677729?
n= 2
Volume of K_B_n= 10.6277921057?
Volume of K_G_n= 11.7653410349?
n= 3
Volume of K_B_n= 11.5575300151?
Volume of K_G_n= 12.8509330423?
n= 4
Volume of K_B_n= 12.1649830025?
Volume of K_G_n= 13.5789109579?
n= 5
Volume of K_B_n= 12.5429092245?
Volume of K_G_n= 14.03716017562?
n= 6
Volume of K_B_n= 12.7761819903?
Volume of K_G_n= 14.32153594738?
n= 7
Volume of K_B_n= 12.9247880752?
Volume of K_G_n= 14.50346127446?
n= 8
Volume of K_B_n= 13.0236549774?
Volume of K_G_n= 14.6250159694?
n= 9
Volume of K_B_n= 13.0922169585?
Volume of K_G_n= 14.7096782388?
n= 10
Volume of K_B_n= 13.1415098025?
Volume of K_G_n= 14.77080444490?
n= 11
Volume of K_B_n= 13.1780501252?
Volume of K_G_n= 14.8162991502?
n= 12
Volume of K_B_n= 13.2058475154?
Volume of K_G_n= 14.85103887133?
n= 13
Volume of K_B_n= 13.2274647902?
Volume of K_G_n= 14.8781496164?
n= 14
Volume of K_B_n= 13.2445965214?
Volume of K_G_n= 14.89970458155?
n= 15
Volume of K_B_n= 

This proves that all the pairs are not isotopic and thus finishes the proof of our theorem.